In [3]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb 14 11:58:40 2023

@author: louis
"""

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sgn
plt.style.use(["science", "notebook"])
%matplotlib tk

# =============================================================================
# Grid and loop variables
# =============================================================================

#Spatial variables of the grid
Ngrid = 111 #Number of spatial grid points
x, dx = np.linspace(0.01, 1, Ngrid, retstep = True)

nu = 1 #Viscosity coefficient

#Fluid velocity
u = -4.5*nu/x #From analytical expression of equation (1) of the problem set 

#outflow boundary conditions
u[0] = -abs(u[1])
u[-1] = abs(u[-2])

#Temporal variables of the simulation 
Nsteps = 1000 #600 #Number of temporal iteration
dt = abs(dx/(max(u)*50)) #Temporal resolution to have stability

beta = 3*nu*dt/dx**2 #Constant from the numerical methods lecture

#Initial condition of the surface density of the accretion disk as a sharp gaussian centered on the peak 
sigma = sgn.gaussian(Ngrid, std = 2) 

#Arrays for the advection and diffusion operators that will be used to update the f array at each temporal iteration 
advec = np.copy(sigma)
diffusion = np.copy(sigma)

# =============================================================================
# Numerical Solver and figure
# =============================================================================

plt.ion()
fig, ax = plt.subplots(1,1)

#Plot of the initial condition
ax.plot(x,sigma, '--', label = 'Initial condition')
ax.set_xlim(0,1)
ax.set_ylim(0,1.05)
ax.set_xlabel("x", weight = 'bold', fontsize = 15)
ax.set_ylabel("$\\mathbf{\\Sigma}$ [A.U.]", fontsize = 15, weight = 'bold')
ax.set_title("Temporal Evolution of the Accretion Disk's Surface Density", fontsize = 14, weight = 'bold')

#Plot to be updated during the animation
pl, = ax.plot(x,sigma, 'r', label = 'Evolution')
ax.legend(frameon = True, edgecolor = 'black', fontsize = 15)

#Draw canvas
fig.canvas.draw()

#Temporal evolution
for i in range(Nsteps):
    
    #Advection operator 
    advec[1:-1] = 0.5*(advec[2:] + advec[:-2] - (u[1:-1]*dt/dx)*(advec[2:] - advec[:-2]))
    
    #Diffusion operator
    A = np.eye(Ngrid)*(1. + 2*beta) + np.eye(Ngrid, k=1)* -beta + np.eye(Ngrid, k=-1)*-beta
    diffusion = np.linalg.solve(A,diffusion)
    
    #Update the function from the contribution of the advection term and the diffusion term
    sigma[1:-1] = (advec[1:-1] + diffusion[1:-1])
    
    #Outflow boundary condition for the surface density
    sigma[0] = sigma[1]
    sigma[-1] = sigma[-2]
        
    #Update data
    pl.set_ydata(sigma)
    fig.canvas.draw()
    plt.pause(0.00001)
    


In [95]:
# -*- coding: utf-8 -*-

import numpy as np
import matplotlib.pyplot as plt
from scipy import special  #Take as input the order and the argument
from scipy import signal
plt.style.use(["science", "notebook"])

# =============================================================================
# Surface density temporal and spatial evolution
# =============================================================================

def surface_density(x,tau):
    sigma = np.exp(-(1+x**2)/tau) * special.iv(0.25, 2*x/tau) / (tau * x**0.25)
    return sigma

# =============================================================================
# Loop for graph
# =============================================================================
    
x = np.arange(0.1, 3, 0.01)

#plt.plot(x, signal.unit_impulse(x.shape[0], 90)/2, label = '$\\tau = 0.0$', color = 'black')

plt.axvline(1, 0, 1, label = '$\\tau = 0.0$', color = 'black')

t = [0.1, 0.25, 0.5, 1.0, 1.5, 2.5, 4.5] #np.arange(0.25,3.5, 0.75)

for i in t: #range(1, 6):
    plt.plot(x, surface_density(x,i), label = "$\\tau = {:}$".format(i))

plt.title("Evolution of surface density", fontsize = 15, weight = 'bold')
plt.xlabel("$\\mathbf{x}$", fontsize = 15)
plt.ylabel("$\\mathbf{\\Sigma / (\\frac{m}{\\pi R_{0}^{2}}})$", fontsize = 15 )

plt.legend(frameon = True, edgecolor = 'black', fontsize = 15 )





